In [1]:
import re
import nltk

import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from nltk.tokenize import word_tokenize
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import Dataset, DataLoader

nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/noble6/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
cd drive/MyDrive/datasets

[Errno 2] No such file or directory: 'drive/MyDrive/datasets'
/content


## 1. Представление и предобработка текстовых данных в виде последовательностей

1.1 Представьте первое предложение из строки `text` как последовательность из индексов слов, входящих в это предложение

In [4]:
text = 'Select your preferences and run the install command. Stable represents the most currently tested and supported version of PyTorch. Note that LibTorch is only available for C++'

In [11]:
from nltk.tokenize import word_tokenize, sent_tokenize

words = word_tokenize(text.lower())
unique_words = set(words)


vocab = {word: i for i, word in enumerate(unique_words)}
print(vocab)

seq_1 = sent_tokenize(text.lower())[0]

words = seq_1.split()
# indexes = [i for i in range(len(words))]

indexes = [vocab[word] for word in word_tokenize(seq_1)]
print(indexes)

{'your': 0, 'and': 1, 'available': 2, '.': 3, 'stable': 4, 'select': 5, 'most': 6, 'version': 7, 'is': 8, 'note': 9, 'c++': 10, 'install': 11, 'pytorch': 12, 'that': 13, 'the': 14, 'run': 15, 'represents': 16, 'tested': 17, 'supported': 18, 'for': 19, 'libtorch': 20, 'currently': 21, 'only': 22, 'command': 23, 'of': 24, 'preferences': 25}
[5, 0, 25, 1, 15, 14, 11, 23, 3]


1.2 Представьте первое предложение из строки `text` как последовательность векторов, соответствующих индексам слов. Для представления индекса в виде вектора используйте унитарное кодирование. В результате должен получиться двумерный тензор размера `количество слов в предложении` x `количество уникальных слов`

In [3]:
text = 'Select your preferences and run the install command. Stable represents the most currently tested and supported version of PyTorch. Note that LibTorch is only available for C++'

In [90]:
word_tokenize(sent_tokenize(text)[0].lower())

['select',
 'your',
 'preferences',
 'and',
 'run',
 'the',
 'install',
 'command',
 '.']

In [95]:
words = word_tokenize(text.lower())
print(words)

unique_words = set(words)
print(unique_words)
print(len(unique_words))


vocab = {word: i for i, word in enumerate(unique_words)}


one_hot = torch.eye(len(vocab))
# print(one_hot)


sentence_vectors = []
for token in word_tokenize(sent_tokenize(text)[0].lower()):
    index = vocab[token]
    vector = one_hot[index]
    sentence_vectors.append(vector)


sentence_vectors = torch.stack(sentence_vectors)


print(sentence_vectors.shape)
print(sentence_vectors)

['select', 'your', 'preferences', 'and', 'run', 'the', 'install', 'command', '.', 'stable', 'represents', 'the', 'most', 'currently', 'tested', 'and', 'supported', 'version', 'of', 'pytorch', '.', 'note', 'that', 'libtorch', 'is', 'only', 'available', 'for', 'c++']
{'pytorch', 'for', 'c++', 'libtorch', 'run', 'the', 'preferences', 'only', 'currently', 'select', 'your', 'tested', 'and', 'install', 'stable', 'supported', 'note', 'most', 'command', 'of', 'represents', 'available', 'is', '.', 'that', 'version'}
26
torch.Size([9, 26])
tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
         0., 0

In [12]:
[word_tokenize(word)[0] for word in sent_tokenize(text)[0].split()]

['Select', 'your', 'preferences', 'and', 'run', 'the', 'install', 'command']

1.3 Решите задачу 1.2, используя модуль `nn.Embedding`

In [73]:
words = word_tokenize(text.lower())

unique_words = set(words)
print(len(unique_words))
print(unique_words)

vocab = {word: idx for idx, word in enumerate(unique_words)}
print(vocab)

# sentence_idxs = [vocab[word_tokenize(word)[0].lower()] for word in sent_tokenize(text)[0].split()]
sentence_idxs = [vocab[word] for word in word_tokenize(seq_1)]
print(len(sentence_idxs))
print(sentence_idxs)

embedding = nn.Embedding(26, len(unique_words))
sentence_vectors = embedding(torch.tensor(sentence_idxs))

print(sentence_vectors.shape)
print(sentence_vectors)

26
{'your', 'and', 'available', '.', 'stable', 'select', 'most', 'version', 'is', 'note', 'c++', 'install', 'pytorch', 'that', 'the', 'run', 'represents', 'tested', 'supported', 'for', 'libtorch', 'currently', 'only', 'command', 'of', 'preferences'}
{'your': 0, 'and': 1, 'available': 2, '.': 3, 'stable': 4, 'select': 5, 'most': 6, 'version': 7, 'is': 8, 'note': 9, 'c++': 10, 'install': 11, 'pytorch': 12, 'that': 13, 'the': 14, 'run': 15, 'represents': 16, 'tested': 17, 'supported': 18, 'for': 19, 'libtorch': 20, 'currently': 21, 'only': 22, 'command': 23, 'of': 24, 'preferences': 25}
9
[5, 0, 25, 1, 15, 14, 11, 23, 3]
torch.Size([9, 26])
tensor([[ 1.0648,  0.1353, -1.0362, -0.3865,  2.3046,  0.0031, -0.8892, -0.4977,
         -0.7643,  0.5279, -0.4276, -1.6377,  0.1731, -0.7164,  1.8232, -0.5074,
          2.5577, -0.8898, -1.6276, -0.3785, -0.7020,  1.1315,  2.0073, -1.0435,
         -0.6373,  0.0110],
        [ 1.2139,  0.3473,  0.2868,  0.0765, -1.1176,  0.1702,  0.9629,  1.3340,
  

## 2. Классификация фамилий по национальности (ConvNet)

Датасет: https://disk.yandex.ru/d/owHew8hzPc7X9Q?w=1

2.1 Считать файл `surnames/surnames.csv`. 

2.2 Закодировать национальности числами, начиная с 0.

2.3 Разбить датасет на обучающую и тестовую выборку

2.4 Реализовать класс `Vocab` (токен = __символ__)
  * добавьте в словарь специальный токен `<PAD>` с индексом 0
  * при создании словаря сохраните длину самой длинной последовательности из набора данных в виде атрибута `max_seq_len`

2.5 Реализовать класс `SurnamesDataset`
  * метод `__getitem__` возвращает пару: <последовательность индексов токенов (см. 1.1 ), номер класса> 
  * длина каждой такой последовательности должна быть одинаковой и равной `vocab.max_seq_len`. Чтобы добиться этого, дополните последовательность справа индексом токена `<PAD>` до нужной длины

2.6. Обучить классификатор.
  
  * Для преобразования последовательности индексов в последовательность векторов используйте `nn.Embedding`. Рассмотрите два варианта: 
    - когда токен представляется в виде унитарного вектора и модуль `nn.Embedding` не обучается
    - когда токен представляется в виде вектора небольшой размерности (меньше, чем размер словаря) и модуль `nn.Embedding` обучается

  * Используйте одномерные свертки и пулинг (`nn.Conv1d`, `nn.MaxPool1d`)
    - обратите внимание, что `nn.Conv1d` ожидает на вход трехмерный тензор размерности `(batch, embedding_dim, seq_len)`

2.7 Измерить точность на тестовой выборке. Проверить работоспособность модели: прогнать несколько фамилий студентов группы через модели и проверить результат. Для каждой фамилии выводить 3 наиболее вероятных предсказания.

### на 12 баллов сделать модель для фамилий с учетом кол-ва вхождений символа

In [24]:
import pandas as pd

surnames = pd.read_csv("data/surnames.csv")
surnames['nationality'], _ = pd.factorize(surnames['nationality'])
print(surnames)

        surname  nationality
0      Woodford            0
1          Coté            1
2          Kore            0
3         Koury            2
4        Lebzak            3
...         ...          ...
10975  Quraishi            2
10976   Innalls            0
10977      Król           12
10978    Purvis            0
10979  Messerli            9

[10980 rows x 2 columns]


In [25]:
from sklearn.model_selection import train_test_split

X = surnames['surname'].str.lower()
y = surnames['nationality']
n_classes = y.nunique()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

In [26]:
class Vocab:
  def __init__(self, data):
    tokens = set()
    max_seq_len = 0
    for item in data:
        max_seq_len = max(max_seq_len, len(item))
        tokens.update(item)

    self.idx_to_token = {0: '<PAD>'}
    self.token_to_idx = {'<PAD>': 0}
    for idx, token in enumerate(tokens, start=1):
        self.idx_to_token[idx] = token
        self.token_to_idx[token] = idx
    self.vocab_len = len(self.idx_to_token)
    self.max_seq_len = max_seq_len
    
vocab = Vocab(X)
print(vocab.vocab_len)
print(vocab.idx_to_token)
print(vocab.max_seq_len)

56
{0: '<PAD>', 1: 'u', 2: 'z', 3: 'ã', 4: 'õ', 5: 'ä', 6: '1', 7: 'm', 8: 'b', 9: 'ì', 10: 'v', 11: 'p', 12: 'ß', 13: 'ó', 14: 'ń', 15: "'", 16: 'ö', 17: 'n', 18: 'l', 19: 'ñ', 20: 'ą', 21: 'è', 22: 'r', 23: 'd', 24: 'á', 25: 'ú', 26: 'y', 27: 'à', 28: 't', 29: 'é', 30: 'q', 31: 'ś', 32: 'x', 33: '-', 34: 'k', 35: 's', 36: 'h', 37: 'c', 38: '/', 39: 'ò', 40: 'ù', 41: 'ü', 42: 'ł', 43: 'ê', 44: 'g', 45: 'w', 46: 'a', 47: ':', 48: 'f', 49: 'i', 50: 'ç', 51: 'ż', 52: 'í', 53: 'j', 54: 'o', 55: 'e'}
17


In [27]:
class SurnamesDataset(Dataset):
  def __init__(self, X, y, vocab: Vocab):
    self.X = X
    self.y = y
    self.vocab = vocab

  def vectorize(self, surname):
    surname_t = torch.zeros(self.vocab.max_seq_len, dtype=torch.int64)
    for i, token in enumerate(surname):
        if i >= self.vocab.max_seq_len:
            break
        surname_t[i] = self.vocab.token_to_idx.get(token, 0)
    return surname_t

  def __len__(self):
    return len(self.X)

  def __getitem__(self, idx):
    surname = self.X.iloc[idx]
    label = self.y.iloc[idx]
    surname_t = self.vectorize(surname)
    return surname_t, label

In [28]:
train_dataset = SurnamesDataset(X_train, y_train, vocab)
test_dataset = SurnamesDataset(X_test, y_test, vocab)

train_dataloader = DataLoader(train_dataset, batch_size=128, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=128, shuffle=False)

In [29]:
train_dataset[1][0]

tensor([22, 46, 45, 18, 49, 17, 44, 35,  0,  0,  0,  0,  0,  0,  0,  0,  0])

In [30]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def train_model(model, train_dataloader, test_dataloader, criterion, optimizer, num_epochs):
    model.to(device)
    train_losses, test_losses = [], []
    
    for epoch in range(num_epochs):
        model.train()
        train_loss, test_loss = 0, 0
        for inputs, labels in train_dataloader:
            x = inputs.to(device)
            y = labels.to(device)
            
            logits = model(x)
            loss = criterion(logits, y)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

            train_loss += loss.item()
    
        # Валидация на val_loader
        model.eval()
        with torch.no_grad():
            for inputs, labels in test_dataloader:
                inputs = inputs.to(device)
                labels = labels.to(device)

                outputs = model(inputs)
                loss = criterion(outputs, labels)

                test_loss += loss.item()
        
        train_losses.append(train_loss/len(train_dataloader))
        test_losses.append(test_loss/len(test_dataloader))

        print(f'Epoch {epoch+1}, Train Loss: {train_losses[-1]:.4f}, Test Loss: {test_losses[-1]:.4f}')

def evaluate_model(model, dataloader):
    model.to(device)
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for surnames, labels in dataloader:
            x = surnames.to(device)
            y = labels.to(device)

            logits = model(x)
            _, predicted = torch.max(logits, 1)
            correct += (predicted == y).sum().item()
            total += y.size(0)

    accuracy = correct / total
    print(f'Test Accuracy: {accuracy:.5f}')

def predict(model, dataset, surname):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)

    model.eval()
    with torch.no_grad():
        vectorized = dataset.vectorize(surname)
        tensor = vectorized.unsqueeze(0).to(device)

        logits = model(tensor)
        
        probs = torch.softmax(logits, dim=1).squeeze()
        # print(f"probs: {probs} size: {probs.shape}")
        top3_probs, top3_indices = torch.topk(probs, k=3)
        print(top3_probs, top3_indices)

        top3_nationalities = _[top3_indices.detach().cpu().numpy()]
        print(f'{surname}: {top3_nationalities[0]} ({top3_probs[0]:.4f}), {top3_nationalities[1]} ({top3_probs[1]:.4f}), {top3_nationalities[2]} ({top3_probs[2]:.4f})')

In [63]:
class SurnameClassifier(nn.Module):
    def __init__(self, vocab_size, embedding_dim, output_dim, use_unit_vectors=False):
        super().__init__()
        # self.use_unit_vectors = use_unit_vectors
        # if use_unit_vectors:
        #     self.embedding = nn.Embedding(vocab_size, embedding_dim)
        #     self.embedding.weight.data = torch.eye(vocab_size)
        #     self.embedding.weight.requires_grad = False
        # else:
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.conv1 = nn.Conv1d(embedding_dim if not use_unit_vectors else vocab_size, 32, kernel_size=3)
        self.pool1 = nn.MaxPool1d(kernel_size=2)
        self.conv2 = nn.Conv1d(32, 64, kernel_size=3)
        self.pool2 = nn.MaxPool1d(kernel_size=2)
        self.flatten = nn.Flatten()
        self.fc = nn.Linear(128, output_dim)

    def forward(self, x):
        embedded = self.embedding(x)
        print(f"embedded shape {embedded.shape}")
        # if self.use_unit_vectors:
        #     embedded = embedded.permute(0, 2, 1)
        # print(self.conv1(embedded).shape)
        conv1_out = self.pool1(torch.relu(self.conv1(embedded)))
        print(f"conv1 + maxpool shape {conv1_out.shape}")
        conv2_out = self.pool2(torch.relu(self.conv2(conv1_out)))
        print(f"conv2 + maxpool shape {conv2_out.shape}")
        flattened = self.flatten(conv2_out)
        print(f"flattened shape {flattened.shape}")
        logits = self.fc(flattened)
        print(f"fc shape: {logits.shape}")
        return logits

In [64]:
vocab.vocab_len

56

In [49]:
model = SurnameClassifier(vocab_size=vocab.vocab_len,
                           embedding_dim=vocab.max_seq_len,
                           output_dim=len(set(y_train)),
                           use_unit_vectors=False)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())

In [50]:
total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Количество обучаемых параметров: {total_params}") 

Количество обучаемых параметров: 11146


In [74]:
model.forward(train_dataset.vectorize("tugolukov").unsqueeze(0).to(device))

embedded shape torch.Size([1, 17, 17])
conv1 shape torch.Size([1, 32, 7])
conv2 shape torch.Size([1, 64, 2])
flattened shape torch.Size([1, 128])
fc shape: torch.Size([1, 18])


tensor([[ 1.5628, -0.9788,  0.4500,  2.8036,  0.8153, -0.9843,  0.7247, -0.0391,
         -0.7686,  0.0865, -0.8290, -0.5450, -1.2702, -0.7551, -2.5267, -2.2920,
         -2.5465, -1.7388]], device='cuda:0', grad_fn=<AddmmBackward0>)

In [161]:
train_model(model, train_dataloader, test_dataloader, criterion, optimizer, 15)

Epoch 1, Train Loss: 0.8597, Test Loss: 0.9468
Epoch 2, Train Loss: 0.8409, Test Loss: 0.9334
Epoch 3, Train Loss: 0.8208, Test Loss: 0.9271
Epoch 4, Train Loss: 0.8006, Test Loss: 0.9142
Epoch 5, Train Loss: 0.7848, Test Loss: 0.9056
Epoch 6, Train Loss: 0.7711, Test Loss: 0.9014
Epoch 7, Train Loss: 0.7531, Test Loss: 0.8887
Epoch 8, Train Loss: 0.7394, Test Loss: 0.8825
Epoch 9, Train Loss: 0.7327, Test Loss: 0.8862
Epoch 10, Train Loss: 0.7167, Test Loss: 0.8760
Epoch 11, Train Loss: 0.7047, Test Loss: 0.8653
Epoch 12, Train Loss: 0.6958, Test Loss: 0.8582
Epoch 13, Train Loss: 0.6811, Test Loss: 0.8590
Epoch 14, Train Loss: 0.6731, Test Loss: 0.8610
Epoch 15, Train Loss: 0.6620, Test Loss: 0.8538


In [162]:
evaluate_model(model, test_dataloader)

Test Accuracy: 0.75000


In [163]:
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score

test_loader = DataLoader(test_dataset, batch_size=len(test_dataset), shuffle=True)
X_batch, y_batch = next(iter(test_loader))
predictions = model(X_batch.to(device)).argmax(dim=1).cpu().detach()
print(classification_report(y_batch, predictions))

              precision    recall  f1-score   support

           0       0.68      0.91      0.78       567
           1       0.00      0.00      0.00        36
           2       0.92      1.00      0.96       346
           3       0.88      0.82      0.85       482
           4       0.88      0.86      0.87       161
           5       0.43      0.81      0.56        36
           6       0.66      0.68      0.67       108
           7       0.38      0.21      0.27        81
           8       0.67      0.05      0.09        41
           9       0.54      0.39      0.45       118
          10       0.73      0.75      0.74        32
          11       0.55      0.54      0.55        57
          12       0.65      0.44      0.52        25
          13       0.67      0.24      0.36        49
          14       0.00      0.00      0.00        15
          15       0.40      0.13      0.20        15
          16       0.00      0.00      0.00        14
          17       0.00    

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## 3. Классификация обзоров на фильмы (ConvNet)

Датасет: https://disk.yandex.ru/d/tdinpb0nN_Dsrg

2.1 Создайте набор данных на основе файлов polarity/positive_reviews.csv (положительные отзывы) и polarity/negative_reviews.csv (отрицательные отзывы). Разбейте на обучающую и тестовую выборку.
  * токен = __слово__
  * данные для обучения в датасете представляются в виде последовательности индексов токенов
  * словарь создается на основе _только_ обучающей выборки. Для корректной обработки ситуаций, когда в тестовой выборке встретится токен, который не хранится в словаре, добавьте в словарь специальный токен `<UNK>`
  * добавьте предобработку текста

2.2. Обучите классификатор.
  
  * Для преобразования последовательности индексов в последовательность векторов используйте `nn.Embedding` 
    - подберите адекватную размерность вектора эмбеддинга: 
    - модуль `nn.Embedding` обучается

  * Используйте одномерные свертки и пулинг (`nn.Conv1d`, `nn.MaxPool1d`)
    - обратите внимание, что `nn.Conv1d` ожидает на вход трехмерный тензор размерности `(batch, embedding_dim, seq_len)`


2.7 Измерить точность на тестовой выборке. Проверить работоспособность модели: придумать небольшой отзыв, прогнать его через модель и вывести номер предсказанного класса (сделать это для явно позитивного и явно негативного отзыва)
* Целевое значение accuracy на валидации - 70+%

In [6]:
with open("data/polarity/positive_reviews.txt") as f:
    positive_reviews = sent_tokenize(f.read())
    
with open("data/polarity/negative_reviews.txt") as f:
    negative_reviews = sent_tokenize(f.read())

In [7]:
print(len(positive_reviews), len(negative_reviews))

6042 5835


In [8]:
reviews_df = pd.DataFrame()

reviews_df["text"] = positive_reviews + negative_reviews
reviews_df["category"] = [1 for i in range(len(positive_reviews))] + [0 for i in range(len(negative_reviews))]

reviews_df

,text,category
0,"simplistic , silly and tedious .",1
1,"it's so laddish and juvenile , only teenage bo...",1
2,exploitative and largely devoid of the depth o...,1
3,[garbus] discards the potential for pathologic...,1
4,a visually flashy but narratively opaque and e...,1
...,...,...
11872,may prove to be [tsai's] masterpiece .,0
11873,mazel tov to a film about a family's joyous li...,0
11874,standing in the shadows of motown is the best ...,0
11875,it's nice to see piscopo again after all these...,0


In [9]:
from nltk.stem import WordNetLemmatizer

def preprocess_text(text):
    text = text.lower()
    text = ''.join([' ' if not char.isalpha() and char not in ['.', ',', '!', '?'] else char for char in text])
    
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in tokens]
    
    preprocessed_text = ' '.join(lemmatized_tokens)
    
    return preprocessed_text

reviews_df["text"] = reviews_df["text"].apply(lambda x: preprocess_text(x))
reviews_df

,text,category
0,"simplistic , silly and tedious .",1
1,"it s so laddish and juvenile , only teenage bo...",1
2,exploitative and largely devoid of the depth o...,1
3,garbus discard the potential for pathological ...,1
4,a visually flashy but narratively opaque and e...,1
...,...,...
11872,may prove to be tsai s masterpiece .,0
11873,mazel tov to a film about a family s joyous li...,0
11874,standing in the shadow of motown is the best k...,0
11875,it s nice to see piscopo again after all these...,0


In [10]:
reviews_df.iloc[9153]["text"]

'if a horror movie s primary goal is to frighten and disturb , then they work spectacularly well .'

In [11]:
from sklearn.model_selection import train_test_split

X = reviews_df['text'].str.lower()
y = reviews_df['category']
n_classes = y.nunique()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
X_train = X_train.reset_index(drop=True)
X_test = X_test.reset_index(drop=True)
y_train = y_train.reset_index(drop=True)
y_test = y_test.reset_index(drop=True)

In [12]:
X[9152:9155]

9152    all comedy is subversive , but this unrelentin...
9153    if a horror movie s primary goal is to frighte...
9154        . . a shiver inducing , nerve rattling ride .
Name: text, dtype: object

In [13]:
X_train[9153]

'any one episode of the soprano would send this ill conceived folly to sleep with the fish .'

In [14]:
X_train[9153]

'any one episode of the soprano would send this ill conceived folly to sleep with the fish .'

In [15]:
class Vocab:
  def __init__(self, data):
    self.idx_to_token = {}
    self.token_to_idx = {}
    self.vocab_len = 0

    # Получаем список всех слов в данных
    all_words = [word for sentence in data for word in word_tokenize(sentence)]
    # print(all_words)

    # Строим словарь
    for word in all_words:
        if word not in self.token_to_idx:
            self.idx_to_token[self.vocab_len] = word
            self.token_to_idx[word] = self.vocab_len
            self.vocab_len += 1



vocab = Vocab(X)
vocab.vocab_len

16453

In [16]:
class Vocab:
  def __init__(self, data):
    self.idx_to_token = {}
    self.token_to_idx = {}
    self.vocab_len = 0
    self.max_seq_len = 0
    
    for item in data:
      self.max_seq_len = max(self.max_seq_len, len(item))
      
    self.idx_to_token = {0: '<UNK>'}
    self.token_to_idx = {'<UNK>': 0}
    self.vocab_len += 1
    # Получаем список всех слов в данных
    all_words = [word for sentence in data for word in word_tokenize(sentence)]
    # print(all_words)

    # Строим словарь
    for word in all_words:
        if word not in self.token_to_idx:
            self.idx_to_token[self.vocab_len] = word
            self.token_to_idx[word] = self.vocab_len
            self.vocab_len += 1

    # Добавляем токен для неизвестных слов
    self.idx_to_token[self.vocab_len] = '<UNK>'
    self.token_to_idx['<UNK>'] = self.vocab_len
    self.vocab_len += 1

    # Преобразуем данные в последовательности индексов токенов
    self.data = []
    for sentence in data:
        tokens = [self.token_to_idx.get(word, self.token_to_idx['<UNK>']) for word in word_tokenize(sentence)]
        self.data.append(tokens)
        
vocab = Vocab(X_train)
vocab.vocab_len

504


14862

In [19]:
class ReviewDataset(Dataset):
  def __init__(self, X, y, vocab: Vocab):
    self.X = X
    self.y = y
    self.vocab = vocab

  def vectorize(self, review):
    '''Генерирует представление отзыва review при помощи бинарного кодирования (см. 1.2)'''
    vec = torch.zeros(self.vocab.max_seq_len, dtype=torch.int64)
    

    for i, word in enumerate(word_tokenize(review)):

      if i >= self.vocab.max_seq_len:
        break

      vec[i] = self.vocab.token_to_idx.get(word, 0)
  
    return vec
    
  def __len__(self):
    return len(self.X)

  def __getitem__(self, idx):
    vec = self.vectorize(self.X[idx])
    label = self.y[idx]
    return vec, label
  

In [20]:
train_dataset = ReviewDataset(X_train, y_train, vocab)
test_dataset = ReviewDataset(X_test, y_test, vocab)

train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [60]:
class ReviewClassifier(nn.Module):
    def __init__(self, vocab_size, embedding_dim, output_dim):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.conv1 = nn.Conv1d(embedding_dim, 64, kernel_size=3)
        
        self.conv2 = nn.Conv1d(64, 128, kernel_size=3)
        # self.pool2 = nn.MaxPool1d(kernel_size=2)
        # self.conv3 = nn.Conv1d(16, 32, kernel_size=3)
        # self.pool3 = nn.MaxPool1d(kernel_size=2)
        self.pool = nn.MaxPool1d(kernel_size=2)
        self.flatten = nn.Flatten()
        self.fc = nn.Linear(124 * 128, output_dim)

    def forward(self, x):
        embedded = self.embedding(x)
        # print(f"embedded shape: {embedded.shape}")
        # print(self.conv1(embedded).shape)
        conv1_out = self.pool(torch.relu(self.conv1(embedded)))
        # print(conv1_out.shape)
        conv2_out = self.pool(torch.relu(self.conv2(conv1_out)))
        # conv3_out = self.pool(torch.relu(self.conv3(conv2_out)))
        # print(conv2_out.shape)
        flattened = self.flatten(conv2_out)
        # print(flattened.shape)
        # print(flattened.shape)
        logits = self.fc(flattened)
        return logits

In [61]:
vocab.max_seq_len

504

In [51]:
model = ReviewClassifier(vocab_size=vocab.vocab_len,
                           embedding_dim=vocab.max_seq_len,
                           output_dim=len(set(y_train)))

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())

In [30]:
total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Количество обучаемых параметров: {total_params}") 

Количество обучаемых параметров: 7643730


In [31]:
train_model(model, train_dataloader, test_dataloader, criterion, optimizer, 10)

Epoch 1, Train Loss: 0.7471, Test Loss: 0.6932
Epoch 2, Train Loss: 0.6929, Test Loss: 0.6923
Epoch 3, Train Loss: 0.6910, Test Loss: 0.6898
Epoch 4, Train Loss: 0.6750, Test Loss: 0.6505
Epoch 5, Train Loss: 0.5657, Test Loss: 0.6046
Epoch 6, Train Loss: 0.4153, Test Loss: 0.6034
Epoch 7, Train Loss: 0.3083, Test Loss: 0.6163
Epoch 8, Train Loss: 0.2137, Test Loss: 0.7052
Epoch 9, Train Loss: 0.1466, Test Loss: 0.8002
Epoch 10, Train Loss: 0.1061, Test Loss: 0.9012


In [32]:
evaluate_model(model, test_dataloader)

Test Accuracy: 0.71212


In [33]:
rating_labels = ["Positive", "Negative"]

def predict(model, dataset, review):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)

    model.eval()
    with torch.no_grad():
        vectorized = dataset.vectorize(review)
        tensor = vectorized.unsqueeze(0).to(device)
        logits = model(tensor)
        probs = torch.softmax(logits, dim=1).squeeze()
        print(probs)
        print(f'{rating_labels[probs.argmax()]} ({probs.max():.4f}), {rating_labels[probs.argmin()]} ({probs.min():.4f}) \n{review}')
        

In [45]:
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score

model.to("cpu")

test_loader = DataLoader(test_dataset, batch_size=len(test_dataset), shuffle=True)
X_batch, y_batch = next(iter(test_loader))
predictions = model(X_batch).argmax(dim=1).cpu().detach()
print(classification_report(y_batch, predictions))

              precision    recall  f1-score   support

           0       0.68      0.78      0.72      1157
           1       0.75      0.65      0.70      1219

    accuracy                           0.71      2376
   macro avg       0.72      0.71      0.71      2376
weighted avg       0.72      0.71      0.71      2376



In [34]:
predict(model, train_dataset, "I had a terrible experience at this restaurant. The staff was rude and the food was overpriced for the quality.")

tensor([0.9501, 0.0499], device='cuda:0')
Positive (0.9501), Negative (0.0499) 
I had a terrible experience at this restaurant. The staff was rude and the food was overpriced for the quality.


In [35]:
predict(model, train_dataset, "This restaurant is simply amazing! The food is delicious and the service is outstanding.")

tensor([9.9980e-01, 1.9507e-04], device='cuda:0')
Positive (0.9998), Negative (0.0002) 
This restaurant is simply amazing! The food is delicious and the service is outstanding.


In [36]:
predict(model, train_dataset, "The menu at this restaurant is very limited and the food is nothing special. I wouldn't go back.")

tensor([0.0539, 0.9461], device='cuda:0')
Negative (0.9461), Positive (0.0539) 
The menu at this restaurant is very limited and the food is nothing special. I wouldn't go back.


In [37]:
predict(model, train_dataset, "I can't say enough good things about this restaurant. It's the perfect place for a romantic dinner or a night out with friends.")

tensor([0.9834, 0.0166], device='cuda:0')
Positive (0.9834), Negative (0.0166) 
I can't say enough good things about this restaurant. It's the perfect place for a romantic dinner or a night out with friends.


In [38]:
predict(model, train_dataset, "I had the best dining experience in this restaurant. The ambiance is perfect and the staff is very friendly.")

tensor([0.9984, 0.0016], device='cuda:0')
Positive (0.9984), Negative (0.0016) 
I had the best dining experience in this restaurant. The ambiance is perfect and the staff is very friendly.
